In [ ]:
'''PACKAGE IMPORTS'''
#For data analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#For data import
import glob
import os

#For datetime 
import datetime


In [ ]:
'''PROBE DATA IMPORT'''
#Soil Data from Gropoint sensors
directory = './Raw Data/GroPoint/'
save_path = '../Figures/soilPlots/'
import_path_frost = "./Raw Data/"
all_files = glob.glob(os.path.join(directory, 'S*.txt'))
bog_files = glob.glob(os.path.join(directory, 'S*.csv'))

allSoil = []
for filename in all_files:
    df = pd.read_csv(filename, 
                    sep = ',',
                    header = None, 
                    names = ['DateTime', 'SensorAddress',
                            'SoilMoist_15cm', 'SoilMoist_30cm', 'SoilMoist_45cm', 
                            'SoilTemp1', 'SoilTemp2', 'SoilTemp3', 'SoilTemp4', 'SoilTemp5', 'SoilTemp6'])
        
    df['SensorName'] = filename[-16:-12]
    df['Watershed'] = filename[-16:-14]

    allSoil.append(df)

allSoil = pd.concat(allSoil, ignore_index=True)

#Precipitation data - update from MN DNR site occasionally, eventually replace with MEF data
precip_directory = './Raw Data/'
precip = pd.read_csv(precip_directory + 'GrandRapids_Precip_MNDNR.csv', 
                     na_values = ['T', 'M'], 
                     parse_dates = ['Date'], 
                     names = ['Date', 'Tmax_F', 'Tmin_F', 'P_in', 'Snow_in', 'SnowDepth_in'], 
                     header = 0, 
                     dtype = {'P_in':float, 'Snow_in':float, 'SnowDepth_in':float})

#Import S2 Forest Met Station Data
met_directory = './Cleaned Data/ATM/'
S2Fmet = pd.read_csv(met_directory + '01_CleanedS2F.csv', 
                    parse_dates = ['TIMESTAMP'])

#Import historical soil moisture data
#infile1  ="https://pasta.lternet.edu/package/data/eml/edi/612/2/9769461f3732c922a9af819587922c86".strip() 
#infile1  = infile1.replace("https://","http://")
                 
#histSoil = pd.read_csv(infile1, skiprows = 1, sep = ",",
#        names = ["DATE", "LOCATION", "d_15", "d_46", "d_76", "d_107", "d_137", "d_168",     
#                    "d_198", "d_229", "d_259", "d_290", "d_320"],
#        parse_dates = ['DATE',])

#Import Soil Texture Data
soilText = pd.read_csv('./Cleaned Data/soilValues_calculated.csv',
                    header = 0,
                    names = ['Label', 'Class', 'Stake', 'Horizon', 'startDepth',
                        'endDepth', 'Structure', 'Consistency', 'Mottled', 'Clay',
                        'Silt', 'Sand', 'Watershed', 'satMatricPot', 'satMoistContent',
                        'slope'])

#For timestamp rounding
dt_format = "%Y-%m-%d %H:%M"

In [ ]:
'''FUNCTIONS'''

def roundTime(dt=None, roundTo=60):
   """Round a datetime object to any time lapse in seconds
   dt : datetime.datetime object, default now.
   roundTo : Closest number of seconds to round to, default 1 minute.
   Author: Thierry Husson 2012 - Use it as you want but don't blame me.
   """
   if dt == None : dt = datetime.datetime.now()
   seconds = (dt.replace(tzinfo=None) - dt.min).seconds
   rounding = (seconds+roundTo/2) // roundTo * roundTo
   return dt + datetime.timedelta(0,rounding-seconds,-dt.microsecond)

def plotMoisture(df, P, met, save_path):
    fig, ax1 = plt.subplots(1, 1, 
                        figsize=(6, 3),
                        layout="constrained")
    
    #Soil Moisture
    ax1.plot(df.DateTime, df.SoilMoist_15cm, '-r',
     label = '15cm')
    ax1.plot(df.DateTime, df.SoilMoist_30cm, '-b',
     label = '30cm')
    ax1.plot(df.DateTime, df.SoilMoist_45cm, '-g',
     label = '45cm')
    ax1.set_ylabel("Soil Moisture, VMC [cm3/cm3]")

    ax12 = ax1.twinx()
    Psnip = snipPrecip(P, min(df.DateTime), max(df.DateTime)) 
    ax12.bar(Psnip.Date, Psnip.P_in, color = 'silver', zorder = -2)  
    ax12.set_ylabel("S2 Forest Precipition [in]")  

    plt.xlabel("Date")
    plt.title(str(df.SensorName[0]))

    plt.xlim(min(df.DateTime), max(df.DateTime))

    ax1.legend()

    #plt.show()
    plt.savefig(save_path + "moistfig" + str(df.SensorName[0]) + ".pdf")
    plt.savefig(save_path + "moistfig" + str(df.SensorName[0]) + ".jpg")

    plt.show()

def plotTemp(df, P, met, save_path):
    fig, ax1 = plt.subplots(1, 1, 
                        figsize=(6, 3),
                        sharex=True,
                        layout="constrained")
    

    #Soil Temperature
    ax1.plot(df.DateTime, df.SoilTemp1,
     label = '5cm')
    ax1.plot(df.DateTime, df.SoilTemp2,
     label = '15cm')
    ax1.plot(df.DateTime, df.SoilTemp3, 
     label = '25cm')
    ax1.plot(df.DateTime, df.SoilTemp4, 
     label = '35cm')
    ax1.plot(df.DateTime, df.SoilTemp5, 
     label = '45cm')
    ax1.plot(df.DateTime, df.SoilTemp6, 
     label = '55cm')
    ax1.set_ylabel("Soil Temperature")

    ax12 = ax1.twinx()
    Psnip = snipPrecip(P, min(df.DateTime), max(df.DateTime)) 
    ax12.bar(Psnip.Date, Psnip.P_in, color = 'silver', zorder = -2)  
    ax12.set_ylabel("S2 Forest Precipition [in]")  

    plt.xlabel("Date")
    plt.title(str(df.SensorName[0]))

    plt.xlim(min(df.DateTime), max(df.DateTime))

    ax1.patch.set_visible(False)

    ax1.legend()

    #plt.show()
    plt.savefig(save_path + "tempfig" + str(df.SensorName[0]) + ".pdf")
    plt.savefig(save_path + "tempfig" + str(df.SensorName[0]) + ".jpg")

    plt.show()

def snipPrecip(P, fir, la):
    #function trims the precip log to the specified dates, filling in NaNs where there is no available data
    #P must be a timeseries containing at least two columns, one with dates and one with precip values
    range = pd.date_range(start = fir.date(), end = la.date())
    return P.set_index('Date').reindex(range).rename_axis('Date').reset_index()
   
def snipTemp(met, fir, la):
    #function trims the precip log to the specified dates, filling in NaNs where there is no available data
    #P must be a timeseries containing at least two columns, one with dates and one with precip values
    print(fir)
    print(la)
    range = pd.date_range(start = fir, end = la, freq = '30min')
    return met.set_index('TIMESTAMP').reindex(range).rename_axis('TIMESTAMP').reset_index()
   
def clipTo(df, date):
    #Remove data before a certain date as the sensor is stabilizing
    #def is a gropoint sensor datafile
    #date is the date to clip to
    return df[df.DateTime > pd.to_datetime(date)].reset_index()

def patch_breakpoints(signal, name, depth, breakpt_dict):
    # patch spikes before & after -- assume that after is offset to match exactly before
    signal_offset = signal.copy() # replicating water table series
    ibefore = breakpt_dict[name][depth]['ibefore']
    iafter = breakpt_dict[name][depth]['iafter']
    fill_option= breakpt_dict[name][depth]['fill_opt']
    # include an option to just keep water table where it is at the end of the recovery & only interpolate the breakpoint
       
    # automatically iterate over spikes to calculate offset and fix
    if len(fill_option) == 0: 
        fill_option = len(ibefore)*[0] # this defaults to interpolating
        
    for i, (ibef, iaft) in enumerate(zip(ibefore, iafter)):
        
        # linearly interpolate between before and after
        if fill_option[i] == 0: 
            iinterp = np.arange(ibef, iaft+1)
            xp = [ibef, iaft]
            fp = [signal_offset[ibef], signal_offset[iaft]]
            signal_offset[iinterp] = np.interp(iinterp, xp, fp)
            
        # patch before and after by calculating offset 
        if fill_option[i] == 1:    
            offset = signal[iaft] - signal[ibef]
            signal_offset[ibef:iaft] = signal_offset[ibef]
            signal_offset[iaft:] = signal_offset[iaft:] - offset

        # just return NaNs
        if fill_option[i] == 2:
            signal_offset[ibef:iaft] = np.nan

    return signal_offset

